In [1]:
%run utils.ipynb

In [ ]:
def is_cylinder (volume_id, layer_id):
    return (((volume_id == 8 or volume_id == 13) and layer_id in [2,4,6,8])
           or (volume_id == 17 and layer_id in [2,4]))

In [28]:
def layer_number (volume_id, layer_id):
    assert (volume_id in [8, 13, 17]), "This volume_id is not in a cylinder!"
    assert (((volume_id == 8 or volume_id == 13) and layer_id in [2,4,6,8])
           or (volume_id == 17 and layer_id in [2,4])), "This layer_id is not in this volume_id"
    diction = {(8,2):1, (8,4):2, (8,6):3, (8,8):4, (13,2):5, (13,4):6, (13,6):7, (13,8):8, (17,2):9, (17,4):10}
    return diction[(volume_id, layer_id)]

In [ ]:
hits, cells, particles, truth_df = load_data_single_event(1000)

In [ ]:
hits.head()

In [ ]:
a = gen_tracks(truth)

In [ ]:
truth_df['dist'] = np.sqrt(truth_df['tx']**2+truth_df['ty']**2+truth_df['tz']**2)
grouped = truth_df.groupby('particle_id')['hit_id','dist']

In [ ]:
b = grouped.apply(lambda:)

In [ ]:
merge_by_hit_ids = hits.merge(truth, how = 'inner', on = 'hit_id')
column_vals = list(merge_by_hit_ids.columns)

In [ ]:
test = merge_by_hit_ids.head(100000)

In [2]:
def get_cylinder_hits(hit_dataframe, truth_dataframe):
    volume_layer_list = [(8,2),(8,4),(8,6),(8,8),(13,2),(13,4),(13,6),(13,8),(17,2),(17,4)]
    temp = [] 
    merge_by_hit_ids = hit_dataframe.merge(truth_dataframe, how = 'inner', on ='hit_id')
    for pair in volume_layer_list:
        volume, layer = pair
        test_data = merge_by_hit_ids[(merge_by_hit_ids['volume_id'].values == volume) & (merge_by_hit_ids['layer_id'].values == layer)]
        temp.append(test_data)
    return pd.concat(temp)    

In [ ]:
def cylinder_batch(max_seq_len, batch_size, feature_len, full_tracks = False):
    all_data = load_dataset('data/train_sample', parts=['hits', 'truth'])
    for data in all_data:
        hit_df, truth_df = data[1], data[2]
        hit_and_truth = get_cylinder_hits(hit_df, truth_df)
        prelim_tracks = gen_tracks(hit_and_truth, hit_and_truth)
        yield prelim_tracks
        

In [4]:
hits, cells, particles, truth_df = load_data_single_event(1000)
hit_and_truth = get_cylinder_hits(hits,truth_df)
prelim_tracks = gen_tracks(hit_and_truth)

In [11]:
info = prelim_tracks.values

In [24]:
count = 0
rows_to_remove = []
for row, track in enumerate(info):
    if len(track) is not 10:
        rows_to_remove.append(row)

result = np.delete(info,rows_to_remove)
    


In [62]:
count = 0
bad_rows = []
for idx, track in enumerate(result):
    compare = [x for x in range(1,11)]
    layer_nums = []
    
    for hit in track:
        volum_id = hits.loc[hits['hit_id']==hit]['volume_id'].item()
        layer_id = hits.loc[hits['hit_id']==hit]['layer_id'].item()
        layer_nums.append(layer_number(volum_id, layer_id))
    if layer_nums != compare:
        bad_rows.append(idx)
ideal_tracks = np.delete(result,bad_rows)


In [66]:
len(result)

593